In [2]:
pwd

'/home/nakyung/projects/BDAIFin/notebooks'

In [4]:
import os
os.chdir("..")

In [15]:
import pandas as pd
import numpy as np

df = pd.read_parquet("data/online/train_oss.parquet")
df_f = df[df["fraud"] == 1].copy()

from OFFLINE.features.client_state import add_client_state_feature
class Cfg:
    client_col = "client_id"

df_f_feat = add_client_state_feature(df_f, Cfg)

from OFFLINE.models.stage2 import Stage2Config, make_stage2_matrix, fit_stage2_hdbscan, save_stage2_package

cfg = Stage2Config(
    label_col="fraud",
    id_cols=("client_id","card_id","merchant_id"),
    use_scaler=True,
    fillna_value=0.0,

    onehot_client_top_mccg_prev=False,

    tune_enabled=True,
    tune_min_cluster_sizes=(20,30,50,80,120),
    tune_min_samples=(None,),          
    two_stage_enabled=True,
    dominant_share_threshold=0.90,
    dominant_label_policy="replace",   

    topn=10,
)

X, feat_cols, scaler = make_stage2_matrix(df_f_feat, cfg)
res = fit_stage2_hdbscan(df_f_feat, X, feat_cols, cfg)

print("dominant:", res["dominant_cluster_id"], "share:", res["dominant_cluster_share"])
res["table"].head(30)


/home/nakyung/.local/lib/python3.10/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/nakyung/.local/lib/python3.10/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/nakyung/.local/lib/python3.10/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/nakyung/.local/lib/python3.10/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/nakyung/.local/lib/python3.10/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed 

dominant: 2 share: 0.9215869311551925


,stage,cluster_id,n,share,top_features_absdiff
0,major,2,7898,0.921587,total_debt | yearly_income | per_capita_income...
1,major,-1,499,0.058226,total_debt | yearly_income | per_capita_income...
2,major,3,66,0.007701,total_debt | yearly_income | per_capita_income...
3,major,1,57,0.006651,total_debt | yearly_income | credit_limit | pe...
4,major,0,50,0.005834,total_debt | credit_limit | per_capita_income ...
5,final,2000,7805,0.910735,total_debt | yearly_income | per_capita_income...
6,final,-1,515,0.060093,total_debt | yearly_income | per_capita_income...
7,final,2001,77,0.008985,credit_limit | yearly_income | per_capita_inco...
8,final,3,66,0.007701,total_debt | yearly_income | per_capita_income...
9,final,1,57,0.006651,total_debt | yearly_income | credit_limit | pe...


In [17]:
res["table"]["top_features_absdiff"]

0     total_debt | yearly_income | per_capita_income...
1     total_debt | yearly_income | per_capita_income...
2     total_debt | yearly_income | per_capita_income...
3     total_debt | yearly_income | credit_limit | pe...
4     total_debt | credit_limit | per_capita_income ...
5     total_debt | yearly_income | per_capita_income...
6     total_debt | yearly_income | per_capita_income...
7     credit_limit | yearly_income | per_capita_inco...
8     total_debt | yearly_income | per_capita_income...
9     total_debt | yearly_income | credit_limit | pe...
10    total_debt | credit_limit | per_capita_income ...
11    yearly_income | total_debt | per_capita_income...
12    credit_limit | total_debt | yearly_income | am...
13    yearly_income | per_capita_income | total_debt...
Name: top_features_absdiff, dtype: object

In [18]:
res

{'labels_major': array([2, 2, 2, ..., 2, 2, 2]),
 'labels_minor': array([0, 0, 0, ..., 0, 0, 0]),
 'labels_final': array([2000, 2000, 2000, ..., 2000, 2000, 2000]),
 'rep_major': {'n': 8570,
  'n_clusters_ex_noise': 4,
  'noise_ratio': 0.05822637106184364,
  'largest_share': 0.9215869311551925,
  'silhouette_non_noise': 0.25717964712023295,
  'score': 0.24586487589212877,
  'picked': {'min_cluster_size': 50, 'min_samples': None}},
 'rep_minor': {'n': 7898,
  'n_clusters_ex_noise': 2,
  'noise_ratio': 0.002025829323879463,
  'largest_share': 0.9882248670549506,
  'silhouette_non_noise': 0.24358684676764517,
  'score': 0.2432166539359424,
  'picked': {'min_cluster_size': 20, 'min_samples': None}},
 'dominant_cluster_id': 2,
 'dominant_cluster_share': 0.9215869311551925,
 'prof_major': {'cluster_counts': {-1: 499, 0: 50, 1: 57, 2: 7898, 3: 66},
  'n_clusters_ex_noise': 4,
  'noise_ratio': 0.05822637106184364,
  'cluster_means': {-1: {'amount': 257.0263977050781,
    'current_age': 56.0761

In [ ]:
import pandas as pd
import numpy as np

# 0) 데이터 로드
df = pd.read_parquet("data/online/train_oss.parquet")
print("raw:", df.shape, "pos_rate:", df["fraud"].mean(), "pos:", df["fraud"].sum())

# 1) client_state 피처 붙이기
from OFFLINE.features.client_state import add_client_state_feature

class Cfg:
    client_col = "client_id"

df_feat = add_client_state_feature(df, Cfg)
print("with client_state:", df_feat.shape)


# 2) Stage2: matrix + hdbscan fit 
from OFFLINE.models.stage2 import (
    Stage2Config,
    make_stage2_matrix,
    fit_stage2_hdbscan,
    save_stage2_package,
)

cfg = Stage2Config(
    label_col="fraud",
    id_cols=("client_id","card_id","merchant_id"),

    # preprocessing
    use_scaler=True,
    fillna_value=0.0,
    drop_feature_cols=("_row_id",),

    # categorical one-hot 
    onehot_client_top_mccg_prev=False,

    # tuning + 2-stage
    tune_enabled=True,
    tune_min_cluster_sizes=(20, 30, 50, 80, 120),
    tune_min_samples=(None,),
    two_stage_enabled=True,
    dominant_share_threshold=0.90,
    dominant_label_policy="replace",

    topn=10,
)

# 3) matrix 생성
X, feat_cols, scaler = make_stage2_matrix(df_feat, cfg)
print("X:", X.shape, "n_feat:", len(feat_cols))

# 4) stage2 fit
res = fit_stage2_hdbscan(df_feat=df_feat, X=X, feat_cols=feat_cols, cfg=cfg)

print("\n=== MAJOR PICKED ===")
print(res["rep_major"])
print("dominant_id:", res["dominant_cluster_id"], "share:", res["dominant_cluster_share"])

if res["rep_minor"] is not None:
    print("\n=== MINOR PICKED ===")
    print(res["rep_minor"])

# 5) 결과 테이블 확인
tbl = res["table"].copy()
tbl = tbl.sort_values(["stage", "share"], ascending=[True, False])
display(tbl.head(30))

# 6) 저장 
out_dir = "../artifacts/stage2_all_v1"
save_stage2_package(out_dir, scaler=scaler, feature_cols=feat_cols, cfg=cfg, result=res)
print("saved to:", out_dir)


raw: (708101, 58) pos_rate: 0.01210279324559632 pos: 8570
with client_state: (708101, 77)
X: (708101, 72) n_feat: 72


/home/nakyung/.local/lib/python3.10/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/nakyung/.local/lib/python3.10/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
